<a href="https://colab.research.google.com/github/oscarnavmac/DistillSum-spa/blob/main/Proyecto_Final_Oscar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


# Transformer para generación de resúmenes.

## Dataset

In [2]:
from datasets import load_dataset

dataset_original = load_dataset("tadeodonegana/samsum-es", split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/14730 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

In [3]:
dataset_fil = dataset_original.filter(lambda sentence: "Texto:" not in sentence["target"]+sentence["text"])
dataset_fil = dataset_fil.rename_column("text", "dialogue")
dataset_fil = dataset_fil.rename_column("target", "summary")

Filter:   0%|          | 0/14730 [00:00<?, ? examples/s]

In [4]:
from datasets import DatasetDict

# 90% train, 10% test + validation
train_testvalid = dataset_fil.train_test_split(test_size=0.1, shuffle=False)
# Split the 10% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5, shuffle=False)
# gather everyone if you want to have a single DatasetDict
dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['dialogue', 'summary'],
        num_rows: 13247
    })
    test: Dataset({
        features: ['dialogue', 'summary'],
        num_rows: 736
    })
    valid: Dataset({
        features: ['dialogue', 'summary'],
        num_rows: 736
    })
})

In [6]:
example_indices = [0, 1]

dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print('INPUT DIALOGUE:')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print('BASELINE HUMAN SUMMARY:')
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE:
Trevor: Estoy muy decepcionado con Amazon
Trevor: Pedí un calendario para mi hermana como regalo de Navidad. Es una sorpresa porque además de lo que ella quiere, quería comprar algo que ella no supiera en absoluto. Pero el estado del paquete :/ Quiero decir, el calendario en sí una vez que lo sacas de la caja está bien, pero eso no está bien.
Trevor: Me siento avergonzado de dar algo tan deteriorado como regalo a alguien.
Elise: oh noo, eso es terrible
Elise: el correo siempre hace esto con mis paquetes
Trevor: Pero sabían que lo estaban enviando al extranjero :( deberían haber hecho... algo
Trevor: Incluso tenía envío prioritario
Elise: deberían haberlo hecho ;(
---------------------------------------------------------------------------------------------------
B

In [7]:
from transformers import AutoModelForSeq2SeqLM

model_name='google/flan-t5-large'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [9]:
for i, index in enumerate([0,1]):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    inputs = tokenizer(dialogue, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Trevor: Estoy muy decepcionado con Amazon
Trevor: Pedí un calendario para mi hermana como regalo de Navidad. Es una sorpresa porque además de lo que ella quiere, quería comprar algo que ella no supiera en absoluto. Pero el estado del paquete :/ Quiero decir, el calendario en sí una vez que lo sacas de la caja está bien, pero eso no está bien.
Trevor: Me siento avergonzado de dar algo tan deteriorado como regalo a alguien.
Elise: oh noo, eso es terrible
Elise: el correo siempre hace esto con mis paquetes
Trevor: Pero sabían que lo estaban enviando al extranjero :( deberían haber hecho... algo
Trevor: Incluso tenía envío prioritario
Elise: deberían haberlo hecho ;(
---------------------------------------------------------------------------------------------------
BAS

In [12]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = f"""
Conversación:

{dialogue}

Resume lo que está pasando:
"""

    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Conversación:

Trevor: Estoy muy decepcionado con Amazon
Trevor: Pedí un calendario para mi hermana como regalo de Navidad. Es una sorpresa porque además de lo que ella quiere, quería comprar algo que ella no supiera en absoluto. Pero el estado del paquete :/ Quiero decir, el calendario en sí una vez que lo sacas de la caja está bien, pero eso no está bien.
Trevor: Me siento avergonzado de dar algo tan deteriorado como regalo a alguien.
Elise: oh noo, eso es terrible
Elise: el correo siempre hace esto con mis paquetes
Trevor: Pero sabían que lo estaban enviando al extranjero :( deberían haber hecho... algo
Trevor: Incluso tenía envío prioritario
Elise: deberían haberlo hecho ;(

Resume lo que está pasando:

---------------------------------------------------------